**Hack AI 2024**

Dataset(s):
* https://www.kaggle.com/datasets/ninadaithal/imagesoasis

In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold, GroupKFold
import pathlib

In [9]:
im_width = 496
im_height = 248
batch_size = 61
n = 86_437

In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path('Data'),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(im_height, im_width),
  batch_size=1
)
val_ds = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path('Data'),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(im_height, im_width),
  batch_size=1
)

ds = train_ds.concatenate(val_ds)

fold = KFold(n_splits=1_417, shuffle=True)

for train, test in fold.split(np.arange(len(ds))):
  train = [x+1 for x in train]
  test = [x+1 for x in test]
  train_ds = tf.data.Dataset.from_tensor_slices([ds.skip(t-1).take(t) for t in train]).flat_map(lambda x: x).map(lambda x, y: (x[0, ...], y[0, ...]))
  test_ds = tf.data.Dataset.from_tensor_slices([ds.skip(t-1).take(t) for t in test]).flat_map(lambda x: x).map(lambda x, y: (x[0, ...], y[0, ...]))
  train_ds = train_ds.take(len(train)).batch(batch_size, drop_remainder=True)
  test_ds = test_ds.take(len(test)).batch(batch_size, drop_remainder=True)

  model = tf.keras.Sequential([
    layers.Rescaling(1./255, input_shape=(im_height, im_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4)
  ])

  model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model.fit(train_ds, epochs=3, steps_per_epoch=1_417)
  scores = model.evaluate(test_ds, verbose=0)
  model.save('models/hai_1')

Found 86437 files belonging to 4 classes.
Using 69150 files for training.
Found 86437 files belonging to 4 classes.
Using 17287 files for validation.
Epoch 1/3
 212/1417 [===>..........................] - ETA: 38:06 - loss: 0.1392 - accuracy: 0.9661

KeyboardInterrupt: 